# Diffumon
**A Denoising Diffusion Probabilistic Model (DDPM) for Image Generation**

This notebook demonstrates checkpoint loading and sample generation.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import os
import torch
from diffumon.diffusion.sampler import p_sampler_to_images
from diffumon.utils import get_device
from diffumon.models.unet import Unet
from PIL.Image import Image as PILImage
import pickle

from diffumon.metrics.plots import combine_images

In [3]:
num_samples = 32
checkpoint_path = "../checkpoints/fashion_mnist_100_epochs.pth"
device = get_device()

Using device: cuda


In [4]:
# Load the trained model
print(f"Loading trained model from {checkpoint_path}...")
with open(checkpoint_path, "rb") as f:
    checkpoint = torch.load(f)
    img_dim = checkpoint.get("img_dim", (1, 28, 28))  # FIXME: unhardcode later
    num_channels = img_dim[0]
    chw_dim = img_dim[1:]

noise_schedule = pickle.loads(checkpoint["noise_schedule"])

model = Unet(
    dim=chw_dim[1],
    num_channels=num_channels,
)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
noise_schedule.to(device)
print("Model loaded.")

Loading trained model from ../checkpoints/fashion_mnist_100_epochs.pth...
Model loaded.


/tmp/ipykernel_864754/1482776095.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f)


In [5]:
# Generate samples
print("Generating samples...")
samples: list[PILImage] = p_sampler_to_images(
    model=model,
    ns=noise_schedule,
    num_samples=num_samples,
    dims=img_dim,
    seed=2048,
    output_dir=None,
    device=device,
)

Generating samples...


In [6]:
combined = combine_images(samples, rows=4, cols=8)

# Plot the combined image and display